#### This notebook shows the proccess of creating dataset with all kanji used by JLPT (from N5 to N2)

Load dataset with kanji used in N5 JLPT

In [262]:
import pandas as pd

sheet1 = pd.read_excel("../data/jlpt_2-5.ods", names=["symbol", "on readings", "kun readings", "meaning"], sheet_name=0, header=None)
sheet2 = pd.read_excel("../data/jlpt_2-5.ods", names=["symbol", "on readings", "kun readings", "meaning"], sheet_name=1, header=None)
sheet3 = pd.read_excel("../data/jlpt_2-5.ods", names=["symbol", "on readings", "kun readings", "meaning"], sheet_name=2, header=None)
sheet4 = pd.read_excel("../data/jlpt_2-5.ods", names=["symbol", "on readings", "kun readings", "meaning"], sheet_name=3, header=None)
init_ds = pd.concat([sheet1, sheet2, sheet3, sheet4])

In [263]:
init_ds = init_ds.dropna(how="all").reset_index(drop=True)

In [264]:
japan = init_ds[init_ds["symbol"].isna()][["on readings", "kun readings"]].rename(columns={"on readings": "on readings (japan)", "kun readings": "kun readings (japan)"})
japan = japan.reset_index(drop=True)

In [265]:
init_ds = init_ds.dropna(subset=["symbol"])
init_ds = init_ds.rename(columns={"on readings": "on readings (latin)", "kun readings": "kun readings (latin)"})
init_ds = init_ds.reset_index(drop=True)

In [266]:
n_ds = pd.concat([init_ds.iloc[:, :2], japan.iloc[:, 0], init_ds.iloc[:, 2], japan.iloc[:, 1], init_ds.iloc[:, 3]], axis=1)
n_ds

,symbol,on readings (latin),on readings (japan),kun readings (latin),kun readings (japan),meaning
0,日,"nichi, jitsu","ニチ, ジツ","hi, -bi, -ka","ひ, -び, -か","day, sun, Japan, counter for days"
1,一,ichi,イチ,hito(tsu),ひと(つ),one
2,国,koku,コク,kuni,くに,country
3,人,"jin, nin",ジン、 ニン,hito,ひと,person
4,年,nen,ネン,toshi,とし,"year, counter for years"
...,...,...,...,...,...,...
986,坊,"bou, bo",ボウ、 ボッ,NaN,NaN,"boy, priest"
987,挟,kyou,キョウ,hasa(mu),はさ(む),pinch; between
988,曇,don,ドン,kumo(ru),くも(る),cloudy weather
989,滴,teki,テキ,"shizuku, shitata(ru)",しずく、 したた(る),drip; drop


Read table wtih JIS X 0208 codec and Unicode.\
I downloaded it from this sourse: http://charset.7jp.net/jis0208.html

In [267]:
jis_to_unicode_ds = pd.read_csv("../data/jis_x_0208_to_unicode.csv", skiprows=6, usecols=[2, 6, 7])
jis_to_unicode_ds.dropna(subset=["UTF-16"], inplace=True)
jis_to_unicode_ds = jis_to_unicode_ds[jis_to_unicode_ds["JIS"] != "JIS"]
jis_to_unicode_ds = jis_to_unicode_ds.rename(columns={"実体(SJIS)": "symbol"})
jis_to_unicode_ds

,JIS,UTF-16,symbol
0,2121,3000,NaN
1,2122,3001,、
2,2123,3002,。
3,2124,FF0C,，
4,2125,FF0E,．
...,...,...,...
7969,7422,69C7,槇
7970,7423,9059,遙
7971,7424,7464,瑤
7972,7425,51DC,凜


Merge table on symbol

In [268]:
ds = pd.merge(n_ds, jis_to_unicode_ds, how="inner", on="symbol")
ds[["symbol", "JIS", "UTF-16", "on readings (latin)", "on readings (japan)", "kun readings (latin)", "kun readings (japan)", "meaning"]]

,symbol,JIS,UTF-16,on readings (latin),on readings (japan),kun readings (latin),kun readings (japan),meaning
0,日,467C,65E5,"nichi, jitsu","ニチ, ジツ","hi, -bi, -ka","ひ, -び, -か","day, sun, Japan, counter for days"
1,一,306C,4E00,ichi,イチ,hito(tsu),ひと(つ),one
2,国,3971,56FD,koku,コク,kuni,くに,country
3,人,3F4D,4EBA,"jin, nin",ジン、 ニン,hito,ひと,person
4,年,472F,5E74,nen,ネン,toshi,とし,"year, counter for years"
...,...,...,...,...,...,...,...,...
986,坊,4B37,574A,"bou, bo",ボウ、 ボッ,NaN,NaN,"boy, priest"
987,挟,3634,631F,kyou,キョウ,hasa(mu),はさ(む),pinch; between
988,曇,465E,66C7,don,ドン,kumo(ru),くも(る),cloudy weather
989,滴,4529,6EF4,teki,テキ,"shizuku, shitata(ru)",しずく、 したた(る),drip; drop


Save dataset

In [269]:
ds.to_csv("../data/kanji_n2-n5.csv", index=False)